In [1]:
# Flatten -> 2 D input was transformed into 1 D
# Dense -> fully connected layers, responsible for generating weights and biases for ML 
# Embedding 

# converts entire string set into a VECTOR matrix! 
# averaged out to get a vector per sentence 

# PATTERN on the averaged vector instead!

In [5]:
%%writefile train_script.py
import tensorflow as tf
from tensorflow import keras


Overwriting train_script.py


In [6]:
%%writefile -a train_script.py

(trainx,trainy), (testx, testy) = tf.keras.datasets.imdb.load_data(num_words=10000)

Appending to train_script.py


In [4]:
trainy[:5]

NameError: ignored

In [ ]:
print(testx[9])

In [7]:
%%writefile -a train_script.py

data = tf.keras.datasets.imdb

Appending to train_script.py


In [ ]:
dir(data)

In [8]:
%%writefile -a train_script.py

reverse_dictionary = data.get_word_index()

Appending to train_script.py


In [ ]:
reverse_dictionary['bye']


In [9]:
%%writefile -a train_script.py

word_index = {word:(encoding+3) for word,encoding in reverse_dictionary.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2  # unknown words
word_index['<UNUSED>'] = 3
olddictionary = { encoding:word  for word,encoding in word_index.items()   }

Appending to train_script.py


In [10]:
%%writefile -a train_script.py

dictionary = {}
for i in range(10000):
  dictionary[i] = olddictionary[i]


Appending to train_script.py


In [ ]:
len(dictionary)

In [ ]:
# NLP keywords-> 
# 0 <PAD> 
# 1 <START>
# 2 <UNK>
# 3 <UNUSED>

# To NORMALIZE-> we make sure all INPUTS are of SAME length
# small sentences will be padded with 0 towards the end
# large sentences will be CHOPPED into shorter length! 

In [11]:
%%writefile -a train_script.py

def decoder(sampleinput):
  return " ".join([dictionary[word] for word in sampleinput])

Appending to train_script.py


In [ ]:
decoder(testx[0])

In [ ]:
testy[0] # 0-> negative sentiment

In [ ]:
testy[41] # 1-> positive sentiment!

# sentiment = f(sentence)


In [ ]:
decoder(testx[41])

In [ ]:
print(len(decoder(testx[41])))
print(len(decoder(testx[42])))
print(len(decoder(testx[43])))
# ALL THE SENTENCES should be made to the same length!
# NEURAL NETWORK INPUT is of FIXED SIZE!

In [12]:
%%writefile -a train_script.py

xtrain_padded = keras.preprocessing.sequence.pad_sequences(trainx, value=0, padding='post',
                                                           truncating='post', maxlen=256)
xtest_padded = keras.preprocessing.sequence.pad_sequences(testx, value=0, padding='post', 
                                                          truncating='post', maxlen=256)


Appending to train_script.py


In [ ]:
print(decoder(trainx[142]))
print(decoder(xtrain_padded[142])) # LONG SENTENCE CHOPPED
print(decoder(trainx[162])) # SHORT SENTENCE PADDED
print(decoder(xtrain_padded[162]))

In [ ]:
print(len(xtrain_padded[0]))
len(xtrain_padded[42])


In [13]:
%%writefile -a train_script.py
# vocab size = 10,000
# hyperparams

HP_vocab_size = 10000
HP_input_dim = 160
# weights generated for embedding -> 10000 X 160 => 160000 
HP_hidden_dim = [64, 128, 256]
HP_output_dim = 2
HP_epochs = 50
HP_initial_lr = 0.01
HP_batch_size = 32


Appending to train_script.py


In [ ]:
# Assume: this was the poorer model

layer1 = keras.layers.Embedding(HP_vocab_size, HP_input_dim)
layer2 = keras.layers.GlobalAveragePooling1D()
layer3 = keras.layers.Dense(HP_hidden_dim[0], activation = tf.nn.relu)
layer4 = keras.layers.Dense(HP_output_dim, activation = tf.nn.softmax)
model1 = keras.models.Sequential([layer1,layer2,layer3,layer4])

In [14]:
%%writefile -a train_script.py

# Assume: this was the better model 

layer1 = keras.layers.Embedding(HP_vocab_size, HP_input_dim)
layer2 = keras.layers.GlobalAveragePooling1D()
layer3 = keras.layers.Dense(HP_hidden_dim[1], activation = tf.nn.relu)
layer4 = keras.layers.Dense(1, activation = tf.nn.sigmoid)
model2 = keras.models.Sequential([layer1,layer2,layer3,layer4])

Appending to train_script.py


In [ ]:
model1.compile(loss=tf.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])


In [15]:
%%writefile -a train_script.py

model2.compile(loss=tf.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])


Appending to train_script.py


In [16]:
%%writefile -a train_script.py

xval = xtest_padded[:10000]
xtest_new = xtest_padded[10000:]
yval = testy[:10000]
ytest_new = testy[10000:]


Appending to train_script.py


In [ ]:
# val dataset would otherwise make the learning slower
# but combined with batch size, we can arrive at a middle 
# ground for speed!

# val-> provides testing WHILE the training is happening!
# scoring-> testing happens AFTER the training!
# 3 datasets-> train_test_split(x,y)-> train, score
#              train_test_plit(trainx,trainy)-> train, val
# 3 datasets-> train, val, score
# train + val- during training
# score - for model.predict and evaluation!

In [ ]:
decoder(xval[29])

In [17]:
%%writefile -a train_script.py

import time

start_time = time.time()
history = model2.fit(xtrain_padded, trainy, epochs=5, batch_size=HP_batch_size, validation_data = (xval, yval), verbose=0) 
end_time = time.time()
# When typically= LOG the metric-> Cloud Logging 
#print('Time Taken = ' + str(end_time-start_time))

Appending to train_script.py


In [ ]:
# VECTORIZATION representing entire sentence 
# vector_math_magic() -> single vector -> fictional_boundary-> 2i-3j
# any vector > 2i-3j should be learnt as pos, other neg!
# AVERAGE out of it!
# layer-> GlobalAveragePooling -> take a window of data-> calculate average-> move to next window!
# Algorithm?
# Calculate mega vectors -> take their average-> learn pattern on these avg via ML!
# learn pattern on these avg via ML -> 2 dense layers so that weights and biases can be generated
# SOFTMAX or SIGMOID-> distribution b/w pos/neg or a single probability!

In [18]:
%%writefile -a train_script.py

predictions = model2.predict(xtest_new)

Appending to train_script.py


In [19]:
%%writefile -a train_script.py

perf = model2.evaluate(xtest_new, ytest_new)
#perf
# instead of printing-> CLoud Logging -> log the metrics 

Appending to train_script.py


In [ ]:
model2.summary()

In [20]:
%%writefile -a train_script.py

model2.save('mymodel.h5')

Appending to train_script.py
